# Réservation d'hôtel avec middleware pour membres prioritaires

Ce notebook démontre le **middleware basé sur les fonctions** en utilisant le Microsoft Agent Framework. Nous partons de l'exemple de workflow conditionnel en ajoutant une couche middleware qui accorde des privilèges spéciaux aux membres prioritaires.

## Ce que vous apprendrez :
1. **Middleware basé sur les fonctions** : Intercepter et modifier les résultats de fonctions
2. **Accès au contexte** : Lire et modifier `context.result` après exécution
3. **Implémentation de la logique métier** : Bénéfices pour les membres prioritaires
4. **Surcharge du résultat** : Modifier les résultats des fonctions selon le statut de l'utilisateur
5. **Même workflow, résultats différents** : Changements de comportement pilotés par le middleware

## Architecture du workflow avec middleware :

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## Différence clé avec le workflow conditionnel :

**Sans middleware** (14-conditional-workflow.ipynb) :
- Paris n’a pas de chambres → Rediriger vers alternative_agent

**Avec middleware** (ce notebook) :
- Utilisateur régulier + Paris → Pas de chambres → Rediriger vers alternative_agent
- Utilisateur prioritaire + Paris → 🌟 Middleware prends le dessus ! → Disponible → Rediriger vers booking_agent

## Prérequis :
- Microsoft Agent Framework installé
- Compréhension des workflows conditionnels (voir 14-conditional-workflow.ipynb)
- Token GitHub ou clé API OpenAI
- Compréhension basique des patterns middleware


In [1]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Étape 1 : Définir les modèles Pydantic pour les sorties structurées

Ces modèles définissent le **schéma** que les agents retourneront. Nous avons ajouté un champ `priority_override` pour suivre quand un middleware modifie le résultat de disponibilité.


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Étape 2 : Définir la base de données des membres prioritaires

Pour cette démonstration, nous allons simuler une base de données des membres prioritaires. En production, cela interrogerait une vraie base de données ou API.

**Membres prioritaires :**
- `alice@example.com` - Membre VIP
- `bob@example.com` - Membre Premium  
- `priority_user` - Compte test


In [3]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## Étape 3 : Créez l’outil de réservation d’hôtel

Comme pour le workflow conditionnel, mais cette fois il sera intercepté par un middleware !


In [4]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Étape 4 : 🌟 Créer un Middleware de Vérification de Priorité (LA FONCTIONNALITÉ CLÉ !)

Ceci est la **fonctionnalité principale** de ce notebook. Le middleware :

1. **Intercepte** l'appel de la fonction hotel_booking
2. **Exécute** la fonction normalement en appelant `next(context)`
3. **Inspecte** le résultat dans `context.result`
4. **Remplace** le résultat si l'utilisateur est prioritaire et qu'aucune chambre n'est disponible
5. **Retourne** le résultat modifié à l'agent

**Modèle clé :**
```python
async def my_middleware(context, next):
    await next(context)  # Exécuter la fonction
    # Maintenant context.result contient la sortie de la fonction
    if some_condition:
        context.result = new_value  # Remplacer !
```


In [5]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## Étape 5 : Définir les fonctions de condition pour le routage

Les mêmes fonctions de condition que celles du workflow conditionnel - elles examinent la sortie structurée pour déterminer le routage.


In [6]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## Étape 6 : Créer un exécuteur d'affichage personnalisé

Même exécuteur que précédemment - affiche la sortie finale du workflow.


In [7]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## Étape 7 : Charger les variables d'environnement

Configurer le client LLM (GitHub Models ou OpenAI).


In [8]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## Étape 8 : Créer des agents IA avec un middleware

**DIFFÉRENCE CLÉ :** Lors de la création de l'agent disponibilité, nous passons le paramètre `middleware` !

C'est ainsi que nous injectons le priority_check_middleware dans la chaîne d'invocation de la fonction de l'agent.


In [9]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Étape 9 : Construire le Workflow

Même structure de workflow qu'auparavant - routage conditionnel basé sur la disponibilité.


In [10]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## Étape 10 : Cas de test 1 - Utilisateur régulier à Paris (Pas de substitution)

Un utilisateur régulier essaie de réserver Paris → Pas de chambres → Redirige vers alternative_agent


In [11]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## Étape 11 : Cas de test 2 - 🌟 Utilisateur prioritaire à Paris (AVEC écrasement !)

Un membre prioritaire tente de réserver Paris → Pas de chambres initialement → 🌟 Le middleware intervient ! → Redirige vers booking_agent

**Ceci est la démonstration clé du pouvoir du middleware !**


In [12]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## Étape 12 : Cas de test 3 - Utilisateur prioritaire à Stockholm (Déjà disponible)

L'utilisateur prioritaire essaie Stockholm → Chambres disponibles → Aucune substitution nécessaire → Dirigé vers booking_agent

Cela montre que le middleware n'agit que lorsque c'est nécessaire !


In [13]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## Points clés et concepts de middleware

### ✅ Ce que vous avez appris :

#### **1. Modèle de middleware basé sur les fonctions**

Le middleware intercepte les appels de fonction en utilisant une simple fonction async :

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Avant l'exécution de la fonction
    print("Intercepting...")
    
    # Exécuter la fonction
    await next(context)
    
    # Après l'exécution de la fonction - inspecter le résultat
    if context.result:
        # Modifier le résultat si nécessaire
        context.result = modified_value
```

#### **2. Accès au contexte et remplacement du résultat**

- `context.function` - Accède à la fonction appelée
- `context.arguments` - Lit les arguments de la fonction
- `context.kwargs` - Accède aux paramètres supplémentaires
- `await next(context)` - Exécute la fonction
- `context.result` - Lit/modifie la sortie de la fonction

#### **3. Implémentation de la logique métier**

Notre middleware implémente des avantages pour les membres prioritaires :
- **Utilisateurs réguliers** : Pas de modifications, flux standard
- **Utilisateurs prioritaires** : Remplacement "pas de disponibilité" → "disponible"
- **Logique conditionnelle** : Ne remplace que si nécessaire

#### **4. Même flux, résultats différents**

La puissance du middleware :
- ✅ Pas de changements dans la structure du flux
- ✅ Pas de changements dans la fonction outil
- ✅ Pas de changements dans la logique conditionnelle de routage
- ✅ Juste du middleware → Comportement différent !

### 🚀 Applications concrètes :

1. **Fonctionnalités VIP/Premium**
   - Remplacer les limites de taux pour les utilisateurs premium
   - Fournir un accès prioritaire aux ressources
   - Débloquer dynamiquement les fonctionnalités premium

2. **Tests A/B**
   - Diriger les utilisateurs vers différentes implémentations
   - Tester de nouvelles fonctionnalités avec des utilisateurs spécifiques
   - Déploiements progressifs de fonctionnalités

3. **Sécurité & Conformité**
   - Auditer les appels de fonction
   - Bloquer les opérations sensibles
   - Faire respecter les règles métier

4. **Optimisation des performances**
   - Mettre en cache les résultats pour certains utilisateurs
   - Sauter les opérations coûteuses si possible
   - Allocation dynamique des ressources

5. **Gestion des erreurs & Reprise**
   - Capturer et traiter les erreurs gracieusement
   - Implémenter une logique de nouvelle tentative
   - Revenir à des implémentations alternatives

6. **Journalisation & Suivi**
   - Suivre les temps d'exécution des fonctions
   - Consigner les paramètres et résultats
   - Surveiller les schémas d'utilisation

### 🔑 Différences clés avec les décorateurs :

| Fonctionnalité | Décorateur | Middleware |
|----------------|------------|------------|
| **Portée** | Fonction unique | Toutes les fonctions de l'agent |
| **Flexibilité** | Fixée à la définition | Dynamique à l'exécution |
| **Contexte** | Limité | Contexte complet de l'agent |
| **Composition** | Plusieurs décorateurs | Pipeline middleware |
| **Connaissance de l’agent** | Non | Oui (accès à l’état de l’agent) |

### 📚 Quand utiliser le middleware :

✅ **Utilisez le middleware lorsque :**
- Vous devez modifier le comportement selon l’état utilisateur/session
- Vous souhaitez appliquer une logique à plusieurs fonctions
- Vous avez besoin d’accéder au contexte au niveau de l’agent
- Vous implémentez des préoccupations transversales (journalisation, authentification, etc.)

❌ **N’utilisez pas le middleware lorsque :**
- Validation simple des entrées (utilisez Pydantic)
- Logique spécifique à une fonction (gardez dans la fonction)
- Modifications ponctuelles (modifiez simplement la fonction)

### 🎓 Modèles avancés :

```python
# Plusieurs middlewares (l'ordre d'exécution est important !)
middleware=[
    logging_middleware,      # Journaux d'abord
    auth_middleware,         # Ensuite vérifie l'authentification
    cache_middleware,        # Ensuite vérifie le cache
    rate_limit_middleware,   # Ensuite limite le débit
    priority_check_middleware  # Enfin vérification de priorité
]

# Exécution conditionnelle du middleware
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modifier le résultat
    else:
        # Ne pas exécuter du tout
        context.result = cached_value
```

### 🔗 Concepts liés :

- **Middleware d’agent** : Intercepte les appels agent.run()
- **Middleware de fonction** : Intercepte les appels aux fonctions outils (ce que nous avons utilisé !)
- **Pipeline middleware** : Chaîne de middleware s’exécutant dans l’ordre
- **Propagation de contexte** : Passage de l’état à travers la chaîne de middleware


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Avertissement** :  
Ce document a été traduit à l’aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d’assurer l’exactitude, veuillez noter que les traductions automatiques peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue native doit être considéré comme la source faisant autorité. Pour les informations critiques, une traduction professionnelle humaine est recommandée. Nous n’assumons aucune responsabilité en cas de malentendus ou d’interprétations erronées résultant de l’utilisation de cette traduction.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
